In [17]:
import pandas as pd

# Load the data, there are GN and GP datasets
file_name = "GP_Dataset_1.xlsx"
# Refer to the model.pdf for the target medicine according to the dataset
#target_medicine = "Trimethoprim/sulfamethoxazole".lower()
df = pd.read_excel(file_name)
df = df.drop(columns=["date_of_submission", "report_issued_date", "id", "type", "vitek_id"])

In [18]:
df = pd.get_dummies(df, columns=["species", "submitted_sample", "bacteria_genus"])
for column in df.columns:
    if "S/I/R_" in str(column):
        df = pd.get_dummies(df, columns=[column])

In [19]:
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score
import numpy as np

In [20]:
# Choosing the features column and the target column
features_col = []
for col in df.columns:
    if "species" in col or "submitted_sample" in col or "bacteria_genus" in col or "S/I/R_" in col:
        features_col.append(col)

X = df[features_col]

allmeds = ["Trimethoprim/sulfamethoxazole","Cefovecin","Marbofloxacin","Amikacin","Amoxicillin/clavulanic acid","Imipenem","Enrofloxacin","Cefalexin","Gentamicin","Nitrofurantoin","Doxycycline"]
bestdepthsngn = [7,8,7,9,3,10,7,6,7,6,9]

ylist = []
for i in allmeds:
    ylist.append(df["ans_" + i.lower()])

modellist = []
total_y_test = []

for i in range(len(allmeds)):
    model = DecisionTreeClassifier(max_depth=bestdepthsngn[i],random_state=42, criterion="gini")
    kf = KFold(n_splits=10, shuffle=True, random_state=42)
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = ylist[i].iloc[train_index], ylist[i].iloc[test_index]
        total_y_test = total_y_test + list(y_test)
        model.fit(X_train,y_train)
        modellist.append(modellist)

print(modellist)

[[...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...]]


In [21]:
import joblib
for i in range(len(allmeds)):
    target_medicine = allmeds[i].replace("/","-")
    model_name = "model_" + target_medicine + ".pkl"
    joblib.dump(modellist,model_name)